**Project Overview**

This notebook simulates player behavior in a blockchain-integrated game to analyze the impact of a new game feature using both frequentist and Bayesian A/B testing methods. It incorporates in-game economy elements (like XP, stunts, and crashes), decentralized features (token spend and NFT purchases), and provides actionable insights for stakeholders.

**Key Metrics Defined**

- ARPU (Average Revenue Per User): Total revenue divided by number of users in the group. Useful for measuring monetization effectiveness.
* NFT Purchase Rate: Share of users who made at least one NFT purchase. Indicates adoption of blockchain features.
- XP (Experience Points): Proxy for player progression and engagement. Higher XP may correlate with long-term retention.
* Stunts and Crashes: Gameplay event counts. Can indicate how engaging or challenging the game mechanics are.
- Token Spent: Amount of in-game currency used (including blockchain tokens). Reflects in-game economic activity.

**1. Imports**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from scipy import stats

**2. Simulate Player Data (Control vs Variant)**

In [ ]:
np.random.seed(42)
n_users = 1000
group = np.random.choice(['A', 'B'], size=n_users, p=[0.5, 0.5])

***2.1 Simulate revenue*** 
* Gamma distribution is used to simulate right-skewed revenue data typical in freemium games.

In [ ]:
revenue_A = np.random.gamma(shape=2.0, scale=0.7, size=(group == 'A').sum())
revenue_B = np.random.gamma(shape=2.3, scale=0.9, size=(group == 'B').sum())

***2.2 Simulate gameplay events (e.g., crashes and stunts)***

In [ ]:
crashes_A = np.random.poisson(3, size=(group == 'A').sum())
stunts_A = np.random.poisson(5, size=(group == 'A').sum())
crashes_B = np.random.poisson(4, size=(group == 'B').sum())
stunts_B = np.random.poisson(7, size=(group == 'B').sum())

***2.3 Simulate blockchain metrics: token usage and NFT purchases***

In [ ]:
tokens_A = np.random.exponential(scale=1.5, size=(group == 'A').sum())
tokens_B = np.random.exponential(scale=2.0, size=(group == 'B').sum())
nft_purchases_A = np.random.binomial(1, 0.05, size=(group == 'A').sum())
nft_purchases_B = np.random.binomial(1, 0.08, size=(group == 'B').sum())


***2.4 Simulate player level progression and XP over time***

In [ ]:
days_played_A = np.random.randint(1, 30, size=(group == 'A').sum())
days_played_B = np.random.randint(1, 30, size=(group == 'B').sum())
xp_A = days_played_A * np.random.normal(50, 10, size=(group == 'A').sum())
xp_B = days_played_B * np.random.normal(55, 12, size=(group == 'B').sum())


***2.5 Build combined dataframe***
* Merge all features into a single player-level dataset.

In [ ]:
data = pd.DataFrame({
    'user_id': np.arange(1, n_users + 1),
    'group': group,
    'revenue': np.zeros(n_users),
    'crashes': np.zeros(n_users),
    'stunts': np.zeros(n_users),
    'tokens_spent': np.zeros(n_users),
    'nft_purchased': np.zeros(n_users),
    'xp': np.zeros(n_users),
    'days_played': np.zeros(n_users)})

* Assign values based on group.

In [ ]:
idx_A = data['group'] == 'A'
idx_B = data['group'] == 'B'
data.loc[idx_A, 'revenue'] = revenue_A
data.loc[idx_B, 'revenue'] = revenue_B
data.loc[idx_A, 'crashes'] = crashes_A
data.loc[idx_B, 'crashes'] = crashes_B
data.loc[idx_A, 'stunts'] = stunts_A
data.loc[idx_B, 'stunts'] = stunts_B
data.loc[idx_A, 'tokens_spent'] = tokens_A
data.loc[idx_B, 'tokens_spent'] = tokens_B
data.loc[idx_A, 'nft_purchased'] = nft_purchases_A
data.loc[idx_B, 'nft_purchased'] = nft_purchases_B
data.loc[idx_A, 'xp'] = xp_A
data.loc[idx_B, 'xp'] = xp_B
data.loc[idx_A, 'days_played'] = days_played_A
data.loc[idx_B, 'days_played'] = days_played_B

* Save generated data

In [ ]:
data.to_csv("data/simulated_player_data.csv", index=False)

**3. Exploratory Data Analysis**

In [ ]:
print(data.groupby('group')[['revenue', 'crashes', 'stunts', 'tokens_spent', 'nft_purchased', 'xp']].agg(['mean', 'std', 'median', 'count']))


**4. Visualizations**

***4.1 Revenue distribution***

In [ ]:
plt.figure(figsize=(12, 5))
sns.boxplot(data=data, x='group', y='revenue', palette='Set2')
plt.title("Revenue Distribution by Group")
plt.show()

***4.2 Revenue density***

In [ ]:
plt.figure(figsize=(12, 5))
sns.kdeplot(data[data.group == 'A']['revenue'], label='Group A', fill=True)
sns.kdeplot(data[data.group == 'B']['revenue'], label='Group B', fill=True)
plt.title("Revenue KDE Plot")
plt.legend()
plt.show()

***4.3 Token usage***

In [ ]:
plt.figure(figsize=(12, 5))
sns.kdeplot(data[data.group == 'A']['tokens_spent'], label='Tokens A', fill=True)
sns.kdeplot(data[data.group == 'B']['tokens_spent'], label='Tokens B', fill=True)
plt.title("Token Spending Distribution")
plt.legend()
plt.show()

***4.4 NFT purchase rate***

In [ ]:
nft_purchase_rate = data.groupby('group')['nft_purchased'].mean() *100
plt.figure(figsize=(8, 6))
sns.barplot(x=nft_purchase_rate.index, y=nft_purchase_rate.values, palette='Set2')
plt.title("NFT Purchase Rate by Group")
plt.ylabel("Purchase Rate (%)")
plt.xlabel("Group")
plt.ylim(0, 100)
plt.show()


***4.5 XP/Progression analysis***

In [ ]:
plt.figure(figsize=(12, 5))
sns.histplot(data, x='xp', hue='group', kde=True, element='step')
plt.title("XP Distribution by Group")
plt.show()

***4.6 Event impact analysis***

In [ ]:
plt.figure(figsize=(12, 5))
sns.scatterplot(data=data, x='crashes', y='revenue', hue='group')
plt.title("Crash Count vs Revenue")
plt.show()

plt.figure(figsize=(12, 5))
sns.scatterplot(data=data, x='stunts', y='revenue', hue='group')
plt.title("Stunts vs Revenue")
plt.show()

**Validity of A/B Split and Causal Inference**
* We assume the A/B group assignment is randomized and independent of player behavior. This allows us to attribute observed differences in outcomes (e.g., revenue, XP, NFT engagement) to the new feature, rather than to underlying player traits.
- To support this assumption, we checked that key baseline variables (e.g., days played, XP) are similarly distributed across groups. In a production setting, we would also validate no bias using pre-experiment checks or covariate balancing methods (e.g., matching, stratification).
* Potential confounders such as player tenure or spending tier should be monitored in future tests and segmented as needed.

**5. Frequentist A/B Test (Welch's t-test)**
* Tests if Group B's average revenue is significantly different from Group A.

In [ ]:
t_stat, p_val = stats.ttest_ind(data[data.group == 'B']['revenue'], data[data.group == 'A']['revenue'], equal_var=False)
print(f"t-stat: {t_stat:.4f}, p-value: {p_val:.6f}")

**Statistical Assumptions**
* The t-test used above assumes the sampling distribution of the mean is approximately normal. Although revenue is skewed, the sample size is large enough for the Central Limit Theorem to apply.
- Bayesian estimation does not require this assumption and provides an intuitive probability-based result (e.g., how likely is Group B > Group A). This makes it useful as a complementary method, especially in high-stakes product decisions.

**6. Bayesian Estimation**

***6.1 Simulate posterior distributions of mean revenue for both groups using Monte Carlo sampling.***

In [ ]:
n_samples = 100000
rev_A = data[data.group == 'A']['revenue']
rev_B = data[data.group == 'B']['revenue']
posterior_A = np.random.normal(rev_A.mean(), rev_A.std() / np.sqrt(len(rev_A)), n_samples)
posterior_B = np.random.normal(rev_B.mean(), rev_B.std() / np.sqrt(len(rev_B)), n_samples)

***6.2 Estimate probability that B is better***

In [ ]:
prob_B_superior = (posterior_B > posterior_A).mean()
print(f"Probability that Group B > Group A: {prob_B_superior:.3f}")

***6.3 Posterior plot***

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(posterior_A, bins=100, alpha=0.5, label='Posterior A')
plt.hist(posterior_B, bins=100, alpha=0.5, label='Posterior B')
plt.axvline(np.mean(posterior_A), color='blue', linestyle='--')
plt.axvline(np.mean(posterior_B), color='orange', linestyle='--')
plt.title("Posterior Distributions of Mean Revenue")
plt.xlabel("Mean Revenue")
plt.ylabel("Frequency")
plt.legend()
plt.show()

**7. Summary & Actionable Insight**

***7.1 Calculate ARPU lift***

In [ ]:
lift = (rev_B.mean() - rev_A.mean()) / rev_A.mean()
print(f"Lift in ARPU from Group A to B: {lift:.2%}")
if p_val < 0.05:
    print("\n- Recommend rollout of Group B layout (statistically significant improvement).")
else:
    print("\n- No significant difference found. Keep testing or revert.")

***7.2 Blockchain engagement insight***

In [ ]:
if nft_purchase_rate['B'] > nft_purchase_rate['A']:
    print("\n- Players in Group B show higher NFT purchase rate — may consider expanding Web3 features.")
else:
    print("\n- No improvement in NFT engagement — investigate UX or education gaps for blockchain features.")